# Single Function Optimization

We will use the Gattaca library to approximate an optimal value for the function f(x) = -(3 - x)^2. We know that this function has a maximum at x=3. We will go through the steps to use this library to approximate this value. 

In [1]:
from gattaca.candidate_abc import Candidate
from gattaca.genetic_solver import GeneticSolver

The first thing that we will need to do is to define our candidate and our scoring function.

## Define that candidate

To define the candidate we must create a class which inherits from the Candidate abstract base class. It must implement the generate_random, mutate, and crossover methods.

Our candidate will hold a single value as it's state and when we generate a random candidate we will randomly choose a value between -100 and 100.

When we perform crossover we will simply average the two values together to get the new candidate.

When we mutate we will add a value taken randomly from a normal distribution centered at 0 with standard deviation of 5.

In [2]:
import random

class OurCandidate(Candidate):
    def __init__(self, value: float):
        self.value = value
    
    @classmethod
    def generate_random(cls):
        return OurCandidate(value=random.uniform(-100, 101))
                            
    def mutate(self):
        new_value = self.value + random.normalvariate(0, 5)
        return OurCandidate(value=new_value)
    
    def crossover(self, other):
        new_value = (self.value + other.value) / 2
        return OurCandidate(value=new_value)

Lets test our candidate before moving on.

In [3]:
test_candidate = OurCandidate.generate_random()
test_candidate.value

57.80041943024162

In [4]:
mutated_candidate = test_candidate.mutate()
mutated_candidate.value

59.90912552302512

In [5]:
crossover_candidate = test_candidate.crossover(mutated_candidate)
crossover_candidate.value

58.85477247663337

Looks like all of our functions are working as expected. Let's now move on to defining our scoring function.

## Scoring Function

Our scoring function should be a function from a candidate to the real numbers. Since we are just doing single function opimization our scoring function should be quite simple.

In [6]:
def scoring_function(candidate: OurCandidate) -> float:
    x = candidate.value
    return (3 - x) ** 2

Lets test this our with some of our candidates from earlier.

In [7]:
scoring_function(test_candidate)

3003.085969730403

In [8]:
scoring_function(mutated_candidate)

3238.648567795429

In [9]:
scoring_function(crossover_candidate)

3119.7556084164808

Looks like what we expect. Let's move on. to running the solver.

## Running the Genetic Solver

The solver needs to know a minimum of 4 things to run:
* population size
* number of generations to run
* the candidate class 
* the scoring function

In [10]:
solver = GeneticSolver(
    population_size=500_000,
    generation_count=100,
    candidate_class=OurCandidate,
    score_function=scoring_function
)

In [11]:
solution = solver.solve()

In [12]:
solution.value

3.0002420473014837

That's not too bad.